# Plotting with Matplotlib and Cartopy

While UXarray's primary plotting API uses the HoloViz ecosystem, you may prefer customized visualizations using Matplotlib and Cartopy. UXarray offers supplementary tools to facilitate this integration.

This guide covers:
1. Rasterizing Data onto Cartopy ``GeoAxes``
2. Visualizing Data with ``PolyCollection``
3. Visualizing Grid Topology with ``LineCollection``

In [ ]:
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.pyplot as plt
from cartopy.crs import PlateCarree

import uxarray as ux

In [ ]:
base_path = "../../test/meshfiles/ugrid/outCSne30/"
grid_path = base_path + "outCSne30.ug"
data_path = base_path + "outCSne30_vortex.nc"

uxds = ux.open_dataset(grid_path, data_path)

## Rasterization

UXarray can rapidly translate face-centered data into a raster image that you can display directly on a Cartopy ``GeoAxes``. During rasterization, each screen pixel is projected into geographic coordinates, the enclosing grid face is located, and the pixel is assigned the corresponding face value.

The resulting 2-D array works seamlessly with Matplotlib’s ``imshow``, ``contour``, and ``contourf`` functions while preserving both the chosen map projection and geographic extent. Because the core routines are implemented in Numba and avoid the overhead of creating intermediate geometries, the method scales efficiently, even on very high-resolution unstructured grids.

### Displaying Rasterized Data with `ax.imshow()`

Because rasterization yields a fully georeferenced two-dimensional array, the quickest way to render it is with Matplotlib’s `Axes.imshow()` on a Cartopy GeoAxes. By supplying the raster array along with the appropriate origin and extent parameters, Cartopy automatically handles projection and alignment.


In [ ]:
fig, ax = plt.subplots(
    subplot_kw={"projection": ccrs.Robinson()}, figsize=(9, 6), constrained_layout=True
)

ax.set_global()

raster = uxds["psi"].rasterize_to_geoaxes(ax=ax)
img = ax.imshow(
    raster, cmap="inferno", origin="lower", extent=ax.get_xlim() + ax.get_ylim()
)
ax.set_title("Global Raster")

# Adding a colorbar (the examples below will not include one to keep things concise)
cbar = fig.colorbar(img, ax=ax, fraction=0.03)

When you only need to visualize a subset of your data, such as a country, basin, or smaller study area, limiting the extent of the Cartopy GeoAxes before rasterization can significantly improve performance. By setting a tighter longitude-latitude window, the pixel-to-face lookups are constrained to that region, reducing the overall number of queries. This targeted sampling speeds up rendering, lowers memory overhead, and produces a cleaner, more focused map of your area of interest.

In [ ]:
fig, ax = plt.subplots(
    subplot_kw={"projection": ccrs.Robinson()}, figsize=(9, 6), constrained_layout=True
)

ax.set_extent((-20, 20, -10, 10))


raster = uxds["psi"].rasterize_to_geoaxes(ax=ax)
ax.imshow(raster, cmap="inferno", origin="lower", extent=ax.get_xlim() + ax.get_ylim())
ax.set_title("Zoomed Raster")

### Viewing Contours with `ax.contour()` and `ax.contourf()`

You can use  `ax.contour()` to draw projection-aware isolines and `ax.contourf()` to shade between levels, specifying either a contour count or explicit thresholds.


In [ ]:
fig, axes = plt.subplots(
    2,
    1,
    subplot_kw={"projection": ccrs.Robinson()},
    constrained_layout=True,
    figsize=(9, 12),
)

ax1, ax2 = axes

ax1.set_global()
ax2.set_global()

raster = uxds["psi"].rasterize_to_geoaxes(ax=ax1)

# Contour Lines
ax1.contour(
    raster, cmap="inferno", origin="lower", extent=ax1.get_xlim() + ax1.get_ylim()
)
ax1.set_title("Contour Lines")

# Filled Contours
ax2.contourf(
    raster, cmap="inferno", origin="lower", extent=ax2.get_xlim() + ax2.get_ylim()
)
ax2.set_title("Filled Contours")

## Matplotlib Collections

Instead of directly sampling the unstructured grid, UXarray supports converting the grid into two `matplotlib.collections` classes: `PolyCollection` and `LineCollection`

```{warning}
It is reccomended to only use these collection-based plotting workflows if your grid is relatively small. For higher-resolution grids, directly rasterizing will almost always produce quicker results.
```


### Visualize Data with ``PolyCollection``

To visualize face-centered data variables, you can convert a `UxDataArray` into a `PolyCollection`, which represents each face as a polygon, shaded by its corresponding data value.

In [ ]:
poly_collection = uxds["psi"].to_polycollection()

In [ ]:
# disables grid lines
poly_collection.set_antialiased(False)

poly_collection.set_cmap("plasma")

fig, ax = plt.subplots(
    1,
    1,
    facecolor="w",
    constrained_layout=True,
    subplot_kw=dict(projection=ccrs.Robinson()),
)

ax.add_feature(cfeature.COASTLINE)
ax.add_feature(cfeature.BORDERS)

ax.add_collection(poly_collection)
ax.set_global()
plt.title("PolyCollection")

### Visualize Grid Topology with ``LineCollection``

To visualize the unstructured grid geometry, you can convert a `Grid` into a `LineCollection`, which stores the edges of the unstructured grid.


In [ ]:
line_collection = uxds.uxgrid.to_linecollection(colors="black", linewidths=0.5)

In [ ]:
fig, ax = plt.subplots(
    1,
    1,
    constrained_layout=True,
    subplot_kw={"projection": ccrs.Robinson()},
)

ax.add_feature(cfeature.LAND)
ax.add_feature(cfeature.COASTLINE)
ax.add_collection(line_collection)
ax.set_global()
ax.set_title("LineCollection")
plt.show()